In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import Model

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Agrega una dimension de canales
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()

    self.con2D_32_K_3_relu = Conv2D(32, kernel_size=3, activation='relu')
    self.maxpool2D_2 = MaxPooling2D(pool_size=(2,2))
                   
    self.con2D_64_K_3_relu = Conv2D(64, kernel_size=3, activation='relu')
    self.maxpool2D_2 = MaxPooling2D(pool_size=(2,2))

    self.flatten = Flatten()

    self.dense_20_relu = Dense(20, activation='relu')
    self.dropout_0_25 = Dropout(0.25)
    self.dense_20_softmax = Dense(10, activation='softmax')


  def call(self, x):
    x = self.con2D_32_K_3_relu(x)
    x = self.maxpool2D_2(x)
    x = self.con2D_64_K_3_relu(x)
    x = self.maxpool2D_2(x)
    x = self.flatten(x)
    x = self.dense_20_relu(x)
    x = self.dropout_0_25(x)
    x = self.dense_20_softmax(x)

    return x

# Crea una instancia del modelo
model = MyModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [36]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, loss: {}, accuracy: {}, test_loss: {}, test_accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reinicia las metricas para el siguiente epoch.
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, loss: 0.17285408079624176, accuracy: 94.8183364868164, test_loss: 0.056170135736465454, test_accuracy: 98.18999481201172
Epoch 2, loss: 0.054895177483558655, accuracy: 98.28333282470703, test_loss: 0.04354828968644142, test_accuracy: 98.54000091552734
Epoch 3, loss: 0.04040894657373428, accuracy: 98.74666595458984, test_loss: 0.04322558268904686, test_accuracy: 98.54000091552734
Epoch 4, loss: 0.030873283743858337, accuracy: 99.04999542236328, test_loss: 0.035119667649269104, test_accuracy: 98.